<a href="https://colab.research.google.com/github/TSANT17/ProyectoPDI/blob/main/Entrenamiento_Generador_MultiEstilo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Dependencias


In [ ]:
## 2. Instalar dependencias necesarias
!pip install mediapipe
!pip install facenet-pytorch
!pip install gradio
!pip install torch torchvision
!pip install scikit-learn
!pip install insightface gradio -q
!pip install onnxruntime --quiet

  Using cached gradio-5.31.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.11.12-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.47.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.

#Librerias


In [ ]:
## 3. Importar librerías
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import cv2
from tqdm import tqdm
from glob import glob
from torchvision import models, transforms
from facenet_pytorch import InceptionResnetV1
import mediapipe as mp
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import save_image
import torch.optim as optim
import gradio as gr
import matplotlib.pyplot as plt
import json
from torchvision.models import vgg19

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Desarrollo

In [ ]:
# Normalizar categorias
# Ruta de entrada y salida

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_input = '/content/drive/My Drive/StyleMatcher/dataset'
root_output = '/content/drive/My Drive/StyleMatcher/dataset_resized'
target_size = (512, 512)

def resize_and_save(input_path, output_path):
    os.makedirs(output_path, exist_ok=True)
    img_paths = glob(os.path.join(input_path, '*'))

    for img_path in tqdm(img_paths, desc=f'Redimensionando {input_path}'):
        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            resized = cv2.resize(img, target_size)
            name = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_path, name), resized)
        except Exception as e:
            print(f"Error con {img_path}: {e}")

def process_dataset(root_input, root_output):
    for phase in ['train', 'val']:
        phase_path = os.path.join(root_input, phase)
        for class_name in os.listdir(phase_path):
            input_folder = os.path.join(phase_path, class_name)
            output_folder = os.path.join(root_output, phase, class_name)
            resize_and_save(input_folder, output_folder)

# Ejecutamos
process_dataset(root_input, root_output)

Redimensionando /content/drive/My Drive/StyleMatcher/dataset/train/anime:  33%|███▎      | 314/950 [01:38<03:18,  3.20it/s]


KeyboardInterrupt: 

In [ ]:
## 4. Dataset personalizado
class StyleMatcherDataset(Dataset):
    def __init__(self, identity_dir, pose_dir, style_dir, target_dir, transform=None):
        self.identity_files = sorted([f for f in os.listdir(identity_dir) if f.endswith('.npy')])
        self.pose_files = sorted([f for f in os.listdir(pose_dir) if f.endswith('.png')])
        self.style_files = sorted([f for f in os.listdir(style_dir) if f.endswith('.npy')])
        self.target_files = sorted([f for f in os.listdir(target_dir) if f.endswith('.jpg') or f.endswith('.png')])

        self.identity_dir = identity_dir
        self.pose_dir = pose_dir
        self.style_dir = style_dir
        self.target_dir = target_dir
        self.transform = transform

    def __len__(self):
            return min(len(self.identity_files), len(self.pose_files), len(self.style_files), len(self.target_files))

    def __getitem__(self, idx):
        identity_vec = np.load(os.path.join(self.identity_dir, self.identity_files[idx]))
        style_vec = np.load(os.path.join(self.style_dir, self.style_files[idx]))

        identity_img = np.tile(identity_vec[:3].reshape(1, 1, 3), (256, 256, 1))
        style_img = np.tile(style_vec[:3].reshape(1, 1, 3), (256, 256, 1))  # ✅

        identity_img = torch.tensor(identity_img, dtype=torch.float32).permute(2, 0, 1)
        style_img = torch.tensor(style_img, dtype=torch.float32).permute(2, 0, 1)

        pose_img = Image.open(os.path.join(self.pose_dir, self.pose_files[idx])).convert("RGB")
        target_img = Image.open(os.path.join(self.target_dir, self.target_files[idx])).convert("RGB")

        if self.transform:
            pose_img = self.transform(pose_img)
            target_img = self.transform(target_img)

        x = torch.cat([identity_img, style_img, pose_img], dim=0)  # [9, 256, 256]
        return x, target_img

In [ ]:
## 5. Modelo generador (U-Net)
class Generator(nn.Module):
    def __init__(self, in_channels=9, out_channels=3):
        super(Generator, self).__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 4, 2, 1),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True)
            )
        self.encoder = nn.Sequential(
            block(in_channels, 64),
            block(64, 128),
            block(128, 256),
            block(256, 512)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
## 6. Función de entrenamiento
def entrenar(model, dataloader, estilo_nombre, epochs=10, lr=1e-4, device='cuda'):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.L1Loss()
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in tqdm(dataloader, desc=f"[{estilo_nombre}] Epoch {epoch+1}/{epochs}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"🎨 {estilo_nombre} - Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")
        torch.save(model.state_dict(), f"/content/drive/MyDrive/StyleMatcher/model_{estilo_nombre}_epoch{epoch+1}.pt")

In [ ]:
# ✅ Configurar dispositivo y cuDNN para máximo rendimiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Usando dispositivo:", device)
torch.backends.cudnn.benchmark = True

# ✅ Transformaciones
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# ✅ Datasets
anime_dataset = StyleMatcherDataset(
    identity_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/anime_inputs",
    pose_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/anime_inputs",
    style_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/anime_styles",
    target_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/targets/anime",
    transform=transform
)

painting_dataset = StyleMatcherDataset(
    identity_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/painting_inputs",
    pose_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/painting_inputs",
    style_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/inputs/painting_styles",
    target_dir="/content/drive/MyDrive/StyleMatcher/dataset_final/targets/painting",
    transform=transform
)

# ✅ DataLoaders con mejoras
anime_loader = DataLoader(anime_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)
painting_loader = DataLoader(painting_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)

# ✅ Modelos
anime_model = Generator()
painting_model = Generator()

# ✅ Entrenamiento
entrenar(anime_model, anime_loader, estilo_nombre="anime", epochs=10, device=device)
entrenar(painting_model, painting_loader, estilo_nombre="painting", epochs=10, device=device)

✅ Usando dispositivo: cuda


[anime] Epoch 1/10: 100%|██████████| 225/225 [03:04<00:00,  1.22it/s]


🎨 anime - Epoch 1, Loss: 0.2194


[anime] Epoch 2/10: 100%|██████████| 225/225 [00:07<00:00, 29.43it/s]


🎨 anime - Epoch 2, Loss: 0.1719


...

🎨 painting - Epoch 8, Loss: 0.2008


[painting] Epoch 9/10: 100%|██████████| 188/188 [00:06<00:00, 28.38it/s]


🎨 painting - Epoch 9, Loss: 0.1974


[painting] Epoch 10/10: 100%|██████████| 188/188 [00:06<00:00, 30.35it/s]

🎨 painting - Epoch 10, Loss: 0.1982


In [ ]:
# === INTERFAZ GRADIO FINAL ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Modelo de Estilo: VGG19 ---
vgg = models.vgg19(pretrained=True).features[:29].eval().to(device)
for p in vgg.parameters():
    p.requires_grad = False

def extract_style(img_tensor):
    layers = [0, 5, 10, 19, 28]
    features = []
    x = img_tensor
    for i, layer in enumerate(vgg):
        x = layer(x)
        if i in layers:
            features.append(x.mean([2, 3]))
    return torch.cat(features, dim=1)

# --- Modelo de Identidad: FaceNet ---
!pip install facenet-pytorch --quiet

from facenet_pytorch import InceptionResnetV1
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# --- Modelo de Estilización ---
class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_dropout=False):
        super().__init__()
        if down:
            self.block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )
        else:
            self.block = nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        if use_dropout:
            self.block.add_module("dropout", nn.Dropout(0.5))

    def forward(self, x):
        return self.block(x)

class StylizerUNet(nn.Module):
    def __init__(self, id_dim=512, style_dim=1472):
        super().__init__()
        self.emb_dim = id_dim + style_dim
        self.embedding_expand = nn.Sequential(
            nn.Linear(self.emb_dim, 512 * 4 * 4),
            nn.ReLU(True)
        )
        self.enc1 = UNetBlock(1, 64, down=True)
        self.enc2 = UNetBlock(64, 128, down=True)
        self.enc3 = UNetBlock(128, 256, down=True)
        self.enc4 = UNetBlock(256, 512, down=True)
        self.enc5 = UNetBlock(512, 512, down=True)
        self.enc6 = UNetBlock(512, 512, down=True)
        self.enc7 = UNetBlock(512, 512, down=True)
        self.middle = nn.Sequential(nn.Conv2d(1024, 512, 3, padding=1), nn.ReLU(True))
        self.dec1 = UNetBlock(512, 512, down=False, use_dropout=True)
        self.dec2 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.dec3 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.dec4 = UNetBlock(1024, 512, down=False)
        self.dec5 = UNetBlock(768, 256, down=False)
        self.dec6 = UNetBlock(384, 128, down=False)
        self.dec7 = UNetBlock(192, 64, down=False)
        self.final = nn.Conv2d(65, 3, 3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, pose_map, identity_vec, style_vec):
        emb = torch.cat([identity_vec, style_vec], dim=1)
        emb = self.embedding_expand(emb).view(-1, 512, 4, 4)
        e1 = self.enc1(pose_map)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        e5 = self.enc5(e4)
        e6 = self.enc6(e5)
        e7 = self.enc7(e6)
        emb_up = F.interpolate(emb, size=e7.shape[2:])
        bottleneck = self.middle(torch.cat([e7, emb_up], dim=1))
        d1 = self.dec1(bottleneck)
        d2 = self.dec2(torch.cat([d1, e6], dim=1))
        d3 = self.dec3(torch.cat([d2, e5], dim=1))
        d4 = self.dec4(torch.cat([d3, e4], dim=1))
        d5 = self.dec5(torch.cat([d4, e3], dim=1))
        d6 = self.dec6(torch.cat([d5, e2], dim=1))
        d7 = self.dec7(torch.cat([d6, e1], dim=1))
        pose_resized = F.interpolate(pose_map, size=d7.shape[2:])
        out = self.final(torch.cat([d7, pose_resized], dim=1))
        return self.tanh(out)

# --- Transformaciones de entrada ---
transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor()
])

def generate_image(pose_img, id_img, style_img, mode):
    # Transformaciones
    pose = transform(Image.fromarray(pose_img).convert("L")).unsqueeze(0).to(device)
    id_tensor = transform(Image.fromarray(id_img).convert("RGB")).unsqueeze(0).to(device)
    style_tensor = transform(Image.fromarray(style_img).convert("RGB")).unsqueeze(0).to(device)

    # Extraer vectores
    identity_vec = facenet(id_tensor)
    style_vec = extract_style(style_tensor)

    # 🔧 Normalización de vectores (muy importante)
    identity_vec = F.normalize(identity_vec, dim=1)
    style_vec = F.normalize(style_vec, dim=1)

    # Cargar modelo
    model = StylizerUNet().to(device)
    if mode == "anime":
        model.load_state_dict(torch.load("/content/model_anime_epoch10.pt", map_location=device))
    else:
        model.load_state_dict(torch.load("/content/model_painting_epoch10.pt", map_location=device))
    model.eval()

    # Inferencia
    with torch.no_grad():
        output = model(pose, identity_vec, style_vec)
        output = output.squeeze(0).cpu().permute(1, 2, 0).clamp(0, 1).numpy()
        return Image.fromarray((output * 255).astype(np.uint8))


# === INTERFAZ ===
gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Image(label="Mapa de Pose (PNG)", type="numpy"),
        gr.Image(label="Imagen de Identidad (PNG)", type="numpy"),
        gr.Image(label="Imagen de Estilo (PNG)", type="numpy"),
        gr.Radio(["anime", "painting"], label="Modelo a Usar")
    ],
    outputs=gr.Image(label="Imagen Estilizada"),
    title="StyleMatcher - Generador de Imágenes",
    description="Sube una imagen de pose, una de identidad y una de estilo. Elige el modelo (anime o pintura) para generar tu imagen estilizada."
).launch(debug=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 196MB/s]


  0%|          | 0.00/107M [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c1e172c9699c9b13d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d3c877aba6ac6c0958.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://8029bcf3006df0a913.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://295cfddaf23698c5a0.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://9894352b4c004cacdf.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://c1e172c9699c9b13d8.gradio.live


##PRUEBA

In [ ]:
# 🔧 Instala face_recognition y dependencias necesarias en Colab
!pip install -q face_recognition gradio opencv-python
!apt-get install -y cmake libboost-all-dev
!pip install -q dlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
anime_style_dir = "/content/drive/MyDrive/StyleMatcher/dataset/train/anime"
painting_style_dir = "/content/drive/MyDrive/StyleMatcher/dataset/train/painting"

In [ ]:
import os
import random
from PIL import Image
import face_recognition
import numpy as np

# Función para obtener imágenes de estilo
def get_style_images(style_type):
    folder = anime_style_dir if style_type == "anime" else painting_style_dir
    return sorted([os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))])

# Función principal de combinación
def combine_face_with_style(user_image, style_path):
    user_np = np.array(user_image.convert("RGB"))
    faces = face_recognition.face_locations(user_np)
    if not faces:
        return user_image, Image.open(style_path), user_image

    top, right, bottom, left = faces[0]
    face_crop = user_np[top:bottom, left:right]
    face_img = Image.fromarray(face_crop).resize((256, 256))
    style_img = Image.open(style_path).convert("RGB").resize((256, 256))
    alpha = 0.5
    blended = Image.blend(face_img, style_img, alpha)
    return user_image, style_img, blended

In [ ]:
import gradio as gr

style_list = get_style_images("anime")

with gr.Blocks() as demo:
    gr.Markdown("## 🎨 Fusionador de Rostro con Estilo Anime o Pintura")

    with gr.Row():
        image_input = gr.Image(label="1️⃣ Imagen del Usuario", type="pil")
        style_selector = gr.Dropdown(label="2️⃣ Estilo a aplicar", choices=style_list, value=style_list[0])
        style_type = gr.Radio(["anime", "painting"], label="🎭 Tipo de estilo", value="anime")

    run_button = gr.Button("✨ Aplicar Estilo")

    with gr.Row():
        original_output = gr.Image(label="🧍 Imagen Original")
        style_output = gr.Image(label="🎨 Estilo Elegido")
        result_output = gr.Image(label="🧬 Resultado Fusionado")

    def update_styles(tipo):
        new_list = get_style_images(tipo)
        return gr.update(choices=new_list, value=new_list[0])

    style_type.change(fn=update_styles, inputs=style_type, outputs=style_selector)
    run_button.click(fn=combine_face_with_style, inputs=[image_input, style_selector],
                     outputs=[original_output, style_output, result_output])

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e49eda27ec84dab8ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


PRUEBA 2

In [ ]:
!git clone https://github.com/nagadomi/animeface-2009.git
%cd animeface-2009
!sudo apt-get install -y libopencv-dev
!make


Cloning into 'animeface-2009'...
remote: Enumerating objects: 2961, done.
remote: Total 2961 (delta 0), reused 0 (delta 0), pack-reused 2961 (from 1)
Receiving objects: 100% (2961/2961), 9.16 MiB | 18.29 MiB/s, done.
Resolving deltas: 100% (1976/1976), done.
/content/animeface-2009
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
make: *** No targets specified and no makefile found.  Stop.


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import face_recognition

def get_style_images(style_type):
    folder = anime_style_dir if style_type == "anime" else painting_style_dir
    return sorted([
        os.path.join(folder, f) for f in os.listdir(folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

def fuse_face_into_style(user_image, style_path):
    # Convertir imágenes
    user_np = np.array(user_image.convert("RGB"))
    style_np = cv2.cvtColor(cv2.imread(style_path), cv2.COLOR_BGR2RGB)

    # Detectar rostro del usuario
    user_faces = face_recognition.face_locations(user_np)
    if not user_faces:
        return user_image, Image.open(style_path), user_image

    u_top, u_right, u_bottom, u_left = user_faces[0]
    face_crop = user_np[u_top:u_bottom, u_left:u_right]
    face_resized = cv2.resize(face_crop, (128, 128))

    # Detectar rostro en la imagen de estilo
    style_faces = face_recognition.face_locations(style_np)
    if not style_faces:
        return user_image, Image.open(style_path), user_image

    s_top, s_right, s_bottom, s_left = style_faces[0]
    center = ((s_left + s_right) // 2, (s_top + s_bottom) // 2)

    # Crear máscara blanca del rostro
    mask = 255 * np.ones(face_resized.shape, face_resized.dtype)

    # Fusión con seamlessClone
    fused = cv2.seamlessClone(face_resized, style_np, mask, center, cv2.NORMAL_CLONE)

    return Image.fromarray(user_np), Image.open(style_path), Image.fromarray(fused)


In [ ]:
import gradio as gr

# Cargar lista inicial de estilos (anime por defecto)
style_list = get_style_images("anime")

# Interfaz Gradio completa
with gr.Blocks() as demo:
    gr.Markdown("## 🎭 Fusión de Rostro con Estilo Artístico")

    with gr.Row():
        image_input = gr.Image(label="📸 Imagen del Usuario", type="pil")
        style_type = gr.Radio(["anime", "painting"], label="🎨 Tipo de Estilo", value="anime")
        style_selector = gr.Dropdown(choices=style_list, label="🖼️ Imagen de Estilo", value=style_list[0] if style_list else None)

    run_btn = gr.Button("✨ Fusionar")

    with gr.Row():
        out_orig = gr.Image(label="1️⃣ Imagen Original")
        out_style = gr.Image(label="2️⃣ Imagen de Estilo")
        out_result = gr.Image(label="3️⃣ Resultado Final")

    # Cambiar lista de estilos al cambiar tipo
    def actualizar_estilos(tipo):
        opciones = get_style_images(tipo)
        return gr.update(choices=opciones, value=opciones[0] if opciones else None)

    style_type.change(fn=actualizar_estilos, inputs=style_type, outputs=style_selector)

    run_btn.click(fn=fuse_face_into_style, inputs=[image_input, style_selector],
                  outputs=[out_orig, out_style, out_result])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://079e05cfc187fba689.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import gradio as gr
import numpy as np
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones
def image_loader(img, max_size=400, shape=None):
    if img.mode != 'RGB':
        img = img.convert("RGB")
    if max(img.size) > max_size:
        size = max_size
    else:
        size = max(img.size)
    if shape is not None:
        size = shape
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])
    image = transform(img).unsqueeze(0)
    return image.to(device)

def im_convert(tensor):
    image = tensor.clone().detach().cpu().squeeze(0)
    image = image.numpy().transpose(1, 2, 0)
    image = image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
    image = np.clip(image, 0, 1)
    return image

# Modelo VGG19
vgg = models.vgg19(pretrained=True).features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

# Extracción de características
def get_features(image, model, layers=None):
    if layers is None:
        layers = {
            '0': 'conv1_1',
            '5': 'conv2_1',
            '10': 'conv3_1',
            '19': 'conv4_1',
            '21': 'conv4_2',
            '28': 'conv5_1'
        }
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    return torch.mm(tensor, tensor.t())

# Función principal de inferencia
def style_transfer(content_img, style_img):
    content = image_loader(content_img)
    style = image_loader(style_img, shape=content.shape[-2:])

    content_features = get_features(content, vgg)
    style_features = get_features(style, vgg)
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

    target = content.clone().requires_grad_(True).to(device)
    optimizer = optim.Adam([target], lr=0.003)

    style_weights = {
        'conv1_1': 1.0,
        'conv2_1': 0.75,
        'conv3_1': 0.2,
        'conv4_1': 0.2,
        'conv5_1': 0.2
    }

    content_weight = 1e4
    style_weight = 1e2

    steps = 200
    for i in range(steps):
        target_features = get_features(target, vgg)
        content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2']) ** 2)

        style_loss = 0
        for layer in style_weights:
            target_feature = target_features[layer]
            target_gram = gram_matrix(target_feature)
            style_gram = style_grams[layer]
            layer_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
            style_loss += layer_loss / (target_feature.shape[1] ** 2)

        total_loss = content_weight * content_loss + style_weight * style_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    return im_convert(target)

# Interfaz Gradio
gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.Image(label="Imagen base (contenido)", type="pil"),
        gr.Image(label="Imagen estilo", type="pil")
    ],
    outputs=gr.Image(label="Resultado estilizado"),
    title="🎨 Style Transfer con VGG19",
    description="Generación de imagen estilizada combinando contenido y estilo. No necesita entrenamiento.",
).launch(debug=True)